In [687]:
%%capture

#Instalando o spotipy
!pip install spotipy

In [688]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import BaggingClassifier
import numpy as np
import pandas as pd
import re
import spotipy
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials

In [689]:
client_credentials_manager = SpotifyClientCredentials(client_id='9230b3a140aa47d5976323e954fdb105', client_secret='49ea33f127304b3cb533030a400ce18c')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Com o novo dataset, como cada música, em muitos casos, têm mais de um gênero e como existem gêneros muito específicos, é necessário juntar alguns gêneros.

In [690]:
pd.options.mode.chained_assignment = None


data = pd.DataFrame(columns = ['artists','name', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','genres'])
data = pd.read_csv('data_generos.csv')
data = data.drop('year', axis=1)
data = data.drop('popularity', axis=1)
data = data.drop('loudness', axis=1)


for i,item in enumerate(data['genres']):
    if item != str:
        item = str(item)
    if "hip hop" in item or 'rap' in item:
        data['genres'][i] = 'hip hop'
    elif "rock" in item:
        data['genres'][i] = 'rock'
    elif "jazz" in item:
        data['genres'][i] = 'jazz'
    elif "pop" in item:
        data['genres'][i] = 'pop'
    elif "metal" in item:
        data['genres'][i] = 'metal'
    elif "punk" in item:
        data['genres'][i] = 'punk'
    elif "country" in item or "folk" in item:
        data['genres'][i] = 'country/folk'
    else:
        data['genres'][i] = 'NaN'
    
data

,artists,name,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence,tempo,duration_ms,genres
0,Chuck Berry,Johnny B. Goode,0.7410,0.534,0.803,0.000061,0.3070,0.0743,0.969,167.983,161560,rock
1,Elvis Presley,Jailhouse Rock,0.4100,0.647,0.582,0.000002,0.0715,0.0755,0.915,167.396,146480,rock
2,Ella Fitzgerald Louis Armstrong,Dream A Little Dream Of Me - Single Version,0.9130,0.443,0.104,0.000000,0.1910,0.1010,0.394,76.497,185160,jazz
3,Elvis Presley,Hound Dog,0.7330,0.357,0.756,0.005050,0.7600,0.0621,0.950,174.797,136027,rock
4,Frank Sinatra,Come Fly With Me - Remastered,0.8450,0.574,0.338,0.000000,0.1650,0.0420,0.493,67.008,199093,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1395,Anne Marie,2002,0.0372,0.697,0.683,0.000000,0.1370,0.1170,0.603,96.133,186987,pop
1396,Zedd Maren Morris Grey,The Middle,0.1710,0.753,0.657,0.000000,0.1120,0.0449,0.437,107.010,184732,pop
1397,Cardi B Bad Bunny J Balvin,I Like It,0.0990,0.816,0.726,0.000000,0.3720,0.1290,0.650,136.048,253390,hip hop
1398,FINNEAS,Let's Fall in Love for the Night,0.8020,0.737,0.408,0.000000,0.1710,0.1040,0.374,127.921,190348,pop


Agora já é possível fazer a Análise Exploratória Inicial:

In [691]:
data_media = data
data_media = data_media.dropna(axis=0, subset=['genres'])
data_media = data.groupby('genres').mean()
data_media = data_media.drop('NaN')
data_media

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence,tempo,duration_ms
genres,,,,,,,,,
country/folk,0.550832,0.677889,0.511333,1.667791e-03,0.258744,0.066478,0.652667,104.994111,166757.555556
hip hop,0.162247,0.707325,0.663837,1.033855e-02,0.193493,0.149903,0.547230,117.776138,224386.546798
jazz,0.796930,0.495727,0.247312,9.572235e-02,0.140927,0.049922,0.398040,108.366609,246935.654545
metal,0.297137,0.407000,0.690667,6.866667e-07,0.157200,0.044567,0.428000,92.207000,265337.666667
pop,0.288440,0.641713,0.633126,1.548216e-02,0.158562,0.069550,0.562523,116.318722,222608.654709
punk,0.017785,0.743000,0.730500,5.096000e-04,0.251000,0.064700,0.612000,93.264000,176827.000000
rock,0.286698,0.553797,0.630914,2.611830e-02,0.168410,0.050506,0.602502,123.209192,233310.360544


Os índices vão ser normalizados para que as diferenças fiquem mais nítidas.

In [692]:
# Normalizando para que as diferenças sejam mais perceptíveis
norm = data_media.apply(lambda x: x/x.max(), axis=0)
norm

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,valence,tempo,duration_ms
genres,,,,,,,,,
country/folk,0.691193,0.912367,0.699977,0.017423,1.000000,0.443471,1.000000,0.852161,0.628473
hip hop,0.203590,0.951985,0.908744,0.108006,0.747814,1.000000,0.838453,0.955904,0.845664
jazz,1.000000,0.667197,0.338552,1.000000,0.544658,0.333026,0.609867,0.879533,0.930647
metal,0.372852,0.547779,0.945471,0.000007,0.607549,0.297302,0.655771,0.748378,1.000000
pop,0.361939,0.863678,0.866702,0.161740,0.612814,0.463967,0.861885,0.944075,0.838964
punk,0.022317,1.000000,1.000000,0.005324,0.970069,0.431611,0.937692,0.756957,0.666423
rock,0.359753,0.745353,0.863674,0.272855,0.650874,0.336923,0.923138,1.000000,0.879296


In [693]:
# Variancias
seletor = VarianceThreshold()
seletor.fit_transform(norm)
variancas = seletor.variances_


rowvar = pd.DataFrame(variancas.reshape(-1, len(variancas)), columns=norm.columns)

norm = norm.append(rowvar).rename({0: 'variancia'})
norm = norm.sort_values(by='variancia', ascending=False, axis=1)
norm

,instrumentalness,acousticness,speechiness,energy,liveness,danceability,valence,duration_ms,tempo
country/folk,0.017423,0.691193,0.443471,0.699977,1.000000,0.912367,1.000000,0.628473,0.852161
hip hop,0.108006,0.203590,1.000000,0.908744,0.747814,0.951985,0.838453,0.845664,0.955904
jazz,1.000000,1.000000,0.333026,0.338552,0.544658,0.667197,0.609867,0.930647,0.879533
metal,0.000007,0.372852,0.297302,0.945471,0.607549,0.547779,0.655771,1.000000,0.748378
pop,0.161740,0.361939,0.463967,0.866702,0.612814,0.863678,0.861885,0.838964,0.944075
punk,0.005324,0.022317,0.431611,1.000000,0.970069,1.000000,0.937692,0.666423,0.756957
rock,0.272855,0.359753,0.336923,0.863674,0.650874,0.745353,0.923138,0.879296,1.000000
variancia,0.108937,0.089060,0.049910,0.043479,0.028587,0.023272,0.018433,0.015585,0.008204


Vou separar o dataset em treinamento e em teste para testar sua precisão com uma classificação Decision Tree.

In [694]:
X = data.drop(columns=['name', 'artists', 'genres',])
y = data.genres

In [695]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=15, random_state=39)

In [696]:
dtc = DecisionTreeClassifier(random_state=39)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=39, splitter='best')

In [697]:
dtc.score(X_test, y_test)

0.5333333333333333

Vou alterar alguns critérios do Decision tree para aumentar a porcentagem de acertos.

In [698]:
dtc = DecisionTreeClassifier(max_features=5, random_state=73, max_leaf_nodes=100, max_depth=7, min_samples_split=10)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=5, max_leaf_nodes=100,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=73, splitter='best')

In [699]:
dtc.score(X_test, y_test)

0.8

Agora, irei usar o Bagging Classifier para tentar aumentar ainda mais a porcentagem de acerto.

In [701]:
# Use our decision tree classifier to create an ensemble, draw 0.8 samples from X to train each estimator
bag = BaggingClassifier(dtc, max_samples=0.8, random_state=39)
bag.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=7,
                                                        max_features=5,
                                                        max_leaf_nodes=100,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=73,
        

In [702]:
bag.score(X_test, y_test)

0.8

Chegamos em 80% de acerto. Vou pesquisar três músicas no Spotify pelo spotipy para testar o classificador.

In [703]:
teste = pd.DataFrame(data=None, columns=['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'], index=None)
musicas = ['ac/dc back in black', 'snoop dogg gin n juice', 'john coltrane in a sentimental mood']

i = 0

for z in musicas:
    
    resultados = sp.search(z)

    track_id = resultados['tracks']['items'][0]['id']
        
    features = sp.audio_features(tracks = [track_id])
    
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    speechiness = features[0]['speechiness']
    acousticness = features[0]['acousticness']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    valence = features[0]['valence']
    tempo = features[0]['tempo']
    duration_ms = features[0]['duration_ms']
            
    colunas = [danceability, energy, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms]
    teste.loc[z] = colunas
    i += 1

teste = teste.apply(lambda x: x/x.max(), axis=0)
seletor = VarianceThreshold()
seletor.fit_transform(teste)
variancas = seletor.variances_
rowvar = pd.DataFrame(variancas.reshape(-1, len(variancas)), columns=teste.columns)   
teste = teste.append(rowvar).rename({0: 'variancia'})
teste = teste.sort_values(by='variancia', ascending=False, axis=1)
teste

,acousticness,instrumentalness,speechiness,liveness,valence,energy,tempo,danceability,duration_ms
ac/dc back in black,0.011752,0.114066,0.250000,0.133981,1.000000,0.820633,1.000000,0.564663,1.000000
snoop dogg gin n juice,0.030449,0.003499,1.000000,1.000000,0.453473,1.000000,0.502665,1.000000,0.827784
john coltrane in a sentimental mood,1.000000,1.000000,0.168085,0.336570,0.369594,0.357562,0.574507,0.845173,0.998959
variancia,0.213001,0.198902,0.140144,0.136797,0.078127,0.073259,0.048172,0.032464,0.006551


In [704]:
for x in range(i):
    print(musicas[x], "=", bag.predict(teste)[x])

ac/dc back in black = rock
snoop dogg gin n juice = hip hop
john coltrane in a sentimental mood = jazz
